# Intento 6

| Score | Estimación _holdout_ | Timestamp |
| --- | --- | --- |
| **0.8622** | 0.8693 | 2022-01-02 00:33:48 UTC |

- _Hyperparameter tuning_.

In [1]:
import pandas as pd
import numpy as np
import time
from itertools import product
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.multioutput import MultiOutputClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
import lightgbm as lgb

# código común
import common.preprocessing as pre
from common.vars import SEED

# display all columns
pd.options.display.max_columns = None

In [2]:
train_features = pd.read_csv('../data/flu_training_set_features.csv', index_col='respondent_id')
train_labels = pd.read_csv('../data/flu_training_set_labels.csv', index_col='respondent_id')
test_features = pd.read_csv('../data/flu_test_set_features.csv', index_col='respondent_id')

In [3]:
def preprocess(train_features, test_features):
  """PREPROCESAMIENTO
  """
  labels = {}
  imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
  _train_features = train_features.astype(str)
  # uso SimpleImputer para "eliminar los nulos", como elimina los nombres de las columnas los ponemos de nuevo
  _train_features = pd.DataFrame(imp.fit_transform(_train_features), index=train_features.index)
  _train_features.columns = train_features.columns

  _all = pd.concat([train_features, test_features])

  # aprendo las etiquetas
  for col in _all.columns:
    labels[col] = LabelEncoder()
    labels[col].fit(_all[col].astype(str))
  
  # aplico el etiquetado
  for col in _train_features.columns:
    _train_features[col] = labels[col].transform(_train_features[col])
  
  _test_features = test_features.astype(str)
  _test_features = pd.DataFrame(imp.transform(_test_features), index=test_features.index)
  _test_features.columns = test_features.columns

  # aplico el mismo etiquetado con los valores de test
  for col in _test_features.columns:
    _test_features[col] = labels[col].transform(_test_features[col])
  
  return _train_features, _test_features

In [4]:
train_features, test_features = preprocess(train_features, test_features)

In [5]:
kf = KFold(n_splits=5, shuffle=True, random_state=SEED)

def cross_validation(model, X, y, cv):
  measures = []
  time_total_0 = time.time()

  for train, test in cv.split(X, y):
    X_train = X.iloc[train, :].values
    X_test = X.iloc[test, :].values
    y_train = y.iloc[train, :].values
    y_test = y.iloc[test, :].values
    time_iter_0 = time.time()
    _model = model.fit(X_train, y_train)
    time_iter = time.time() - time_iter_0
    preds = _model.predict_proba(X_test)
    y_preds = pd.DataFrame(
      {
        'h1n1_vaccine':     preds[0][:,1],
        'seasonal_vaccine': preds[1][:,1],
      },
      index = y.iloc[test,:].index
    )
    measure = roc_auc_score(y_test, y_preds)
    print(f"[CV] AUC score: {measure:.4f}. Time: {time_iter:6.2f}")
    measures.append(measure)
  
  time_total = time.time() - time_total_0
  print(f"[CV] Average AUC: {np.mean(measures):.4f}. Total time: {time_total:6.2f}")
  return model, y_preds

def preds_to_df(preds, features):
  return pd.DataFrame(
    {
      'h1n1_vaccine': preds[0][:,1],
      'seasonal_vaccine': preds[1][:,1]
    },
    index = features.index
  )

### _Holdout_ como proxy de ROC en test

In [6]:
def holdout_proxy(model, X, y, test_size=0.2):
  # split train features in two
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=SEED)

  model_cvd, _ = cross_validation(model, X_train, y_train, kf)

  # aprendemos con todos los ejemplos
  model_cvd = model_cvd.fit(X_train, y_train)
  # aplicamos probabilidad
  preds = model_cvd.predict_proba(X_test)

  return roc_auc_score(y_test, preds_to_df(preds, X_test))

In [7]:
def proxy_search(search_space, model_lambda, X, y, test_size=0.2):
  space = (dict(zip(search_space.keys(), values)) for values in product(*search_space.values()))
  n = len(list(product(*search_space.values())))
  results = []
  i = 1
  for params in space:
    print(f"[Proxy Search] ({i} of {n}) Searching with params {params}")
    model = model_lambda(params)
    roc_auc = holdout_proxy(model, X, y, test_size=test_size)
    results.append({
      'params': params,
      'roc_auc': roc_auc
    })
    i += 1
  
  return results

In [8]:
lgbm_multi_lambda = lambda params: MultiOutputClassifier(lgb.LGBMClassifier(objective='binary', **params))

In [9]:
search_space = {
  'n_estimators': [50, 200],
  'colsample_bytree': [i/10.0 for i in range(3, 6)],
  'learning_rate': [0.05, 0.1],
  'num_leaves': [30, 50]
}

In [12]:
search = proxy_search(search_space, lgbm_multi_lambda, train_features, train_labels)

[Proxy Search] (1 of 24) Searching with params {'n_estimators': 50, 'colsample_bytree': 0.3, 'learning_rate': 0.05, 'num_leaves': 30}
[CV] AUC score: 0.8627. Time:   0.30
[CV] AUC score: 0.8643. Time:   0.27
[CV] AUC score: 0.8554. Time:   0.21
[CV] AUC score: 0.8517. Time:   0.20
[CV] AUC score: 0.8676. Time:   0.23
[CV] Average AUC: 0.8603. Total time:   1.35
[Proxy Search] (2 of 24) Searching with params {'n_estimators': 50, 'colsample_bytree': 0.3, 'learning_rate': 0.05, 'num_leaves': 50}
[CV] AUC score: 0.8637. Time:   0.43
[CV] AUC score: 0.8649. Time:   0.64
[CV] AUC score: 0.8557. Time:   0.37
[CV] AUC score: 0.8528. Time:   0.45
[CV] AUC score: 0.8680. Time:   0.41
[CV] Average AUC: 0.8610. Total time:   2.48
[Proxy Search] (3 of 24) Searching with params {'n_estimators': 50, 'colsample_bytree': 0.3, 'learning_rate': 0.1, 'num_leaves': 30}
[CV] AUC score: 0.8660. Time:   0.36
[CV] AUC score: 0.8671. Time:   0.27
[CV] AUC score: 0.8581. Time:   0.29
[CV] AUC score: 0.8551. Time

In [14]:
search_max = max(search, key=lambda item: item['roc_auc'])
search_max

{'params': {'n_estimators': 200,
  'colsample_bytree': 0.5,
  'learning_rate': 0.05,
  'num_leaves': 50},
 'roc_auc': 0.8693384996292246}

In [15]:
# split train features in two
X_train, X_test, y_train, y_test = train_test_split(train_features, train_labels, test_size=0.2, random_state=SEED)

In [16]:
# LightGBM
lgbm = lgb.LGBMClassifier(objective='binary', **search_max['params'])
multi_lgbm = MultiOutputClassifier(lgbm)
lgbm_cvd, y_preds_lgbm = cross_validation(multi_lgbm, X_train, y_train, kf)

[CV] AUC score: 0.8674. Time:   1.21
[CV] AUC score: 0.8678. Time:   1.50
[CV] AUC score: 0.8602. Time:   1.54
[CV] AUC score: 0.8562. Time:   1.41
[CV] AUC score: 0.8734. Time:   1.40
[CV] Average AUC: 0.8650. Total time:   7.45


In [17]:
# aprendemos con todos los ejemplos
lgbm_cvd = lgbm_cvd.fit(X_train, y_train)
# aplicamos probabilidad
lgbm_preds = lgbm_cvd.predict_proba(X_test)

In [18]:
roc_auc_score(y_test, preds_to_df(lgbm_preds, X_test))

0.8693384996292246

### Submission

In [19]:
# LightGBM
lgbm = lgb.LGBMClassifier(objective='binary', **search_max['params'])
multi_lgbm = MultiOutputClassifier(lgbm)
lgbm_cvd, y_preds_lgbm = cross_validation(multi_lgbm, train_features, train_labels, kf)

[CV] AUC score: 0.8693. Time:   1.04
[CV] AUC score: 0.8677. Time:   0.91
[CV] AUC score: 0.8672. Time:   0.98
[CV] AUC score: 0.8707. Time:   1.72
[CV] AUC score: 0.8652. Time:   1.51
[CV] Average AUC: 0.8680. Total time:   6.55


In [20]:
# aprendemos con todos los ejemplos
lgbm_cvd = lgbm_cvd.fit(train_features, train_labels)
# aplicamos probabilidad
lgbm_preds = lgbm_cvd.predict_proba(test_features)

In [21]:
preds_to_df(lgbm_preds, test_features).to_csv('../submissions/06.csv')